In [4]:
import json
import os
import glob
from datetime import datetime

# Directory containing all Slack JSON files
json_folder = "/Users/shukla03/Downloads/help-reverb" 
merged_data = []

# Check if the directory exists
if not os.path.exists(json_folder):
    raise FileNotFoundError(f"The directory {json_folder} does not exist.")

# Loop through all JSON files in the folder
for file_path in glob.glob(os.path.join(json_folder, "*.json")):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            slack_data = json.load(file)

            for message in slack_data:
                if 'replies' in message:  # Only process messages with replies
                    merged_data.append({
                        'query': message['text'],
                        'answers': [reply['text'] for reply in message['replies']],  # All replies
                        'timestamp': message['ts'],
                        'date': datetime.utcfromtimestamp(float(message['ts'].split('.')[0])).strftime('%Y-%m-%d'),
                        'source_file': os.path.basename(file_path)  # Track which file it came from
                    })
    except (json.JSONDecodeError, OSError) as e:
        print(f"Error processing file {file_path}: {e}")

# Sort data by timestamp (oldest to newest)
merged_data.sort(key=lambda x: x['timestamp'])

# Save as a combined JSON file
try:
    with open("merged_slack_data.json", "w", encoding="utf-8") as output_file:
        json.dump(merged_data, output_file, indent=4)
    print("Merged Slack data saved to merged_slack_data.json")
except OSError as e:
    print(f"Error saving merged data: {e}")

KeyError: 'text'